In [2]:
import os
from dotenv import load_dotenv

load_dotenv('../.env')

model_name = os.getenv("LLM_MODEL") or "gpt-4o-mini"
model_provider = os.getenv("LLM_MODEL_PROVIDER") or "openai"

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [3]:
llm = ChatOpenAI(model=model_name, temperature=0.7)

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt1 = PromptTemplate(
    input_variables=["review"],
    template="다음 숙박 시설 리뷰를 한글로 번역하세요.\n\n{review}"
)

prompt2 = PromptTemplate.from_template(
    "다음 숙박 시설 리뷰를 한 문장으로 요약하세요.\n\n{translation}"
)

prompt3 = PromptTemplate.from_template(
    "다음 숙박 시설 리뷰를 읽고 0점부터 10점 사이에서 긍정/부정 점수를 매기세요. 숫자로만 대답하세요.\n\n{summary}"
)

prompt4 = PromptTemplate.from_template(
    "다음 숙박 시설 리뷰에 사용된 언어가 무엇인가요? 언어 이름만 대답하세요.\n\n{review}"
)

prompt5 = PromptTemplate.from_template(
    "다음 숙박 시설 리뷰 요약에 대해 공손한 답변을 작성하세요.\n답변 언어:{language}\n리뷰 요약:{summary}"
)

prompt6 = PromptTemplate.from_template(
    "다음 생성된 답변을 한국어로 번역해주세요.\n 리뷰 번역:{reply}"
)

translate_chain_component = prompt1 | llm | StrOutputParser()
summarize_chain_component = prompt2 | llm | StrOutputParser()
sentiment_score_chain_component = prompt3 | llm | StrOutputParser()
language_chain_component = prompt4 | llm | StrOutputParser()
reply1_chain_component = prompt5 | llm | StrOutputParser()
reply2_chain_component = prompt6 | llm | StrOutputParser()

In [10]:
combined_lecl_chain = (
    RunnablePassthrough.assign(
        translation=lambda x: translate_chain_component.invoke({"review": x["review"]}),
    )
    | RunnablePassthrough.assign(
        summary=lambda x: summarize_chain_component.invoke({"translation": x["translation"]}),
        sentiment_score=lambda x: sentiment_score_chain_component.invoke({"translation": x["translation"]}),
        language=lambda x: language_chain_component.invoke({"review": x["review"]}),
    )
    | RunnablePassthrough.assign(
        reply1=lambda x: reply1_chain_component.invoke({"summary": x["summary"], "language": x["language"]}),
    )
    | RunnablePassthrough.assign(
        reply2=lambda x: reply2_chain_component.invoke({"reply": x["reply1"]}),
    )
)   

In [7]:
review_text = """
The hotel was clean and the staff were very helpful.
The location was convenient, close to my attractions.
However, the room was a bit small and the breakfast options were limited.
Overall, a decent stay but there is room for improvement.
"""


In [11]:
# 체인 실행 및 결과 출력
try:
    result = combined_lecl_chain.invoke({"review": review_text})
    
    print(f'translation 결과 \n{result.get("translation", "N/A")}\n')
    print(f'summary 결과 \n{result.get("summary", "N/A")}\n')
    print(f'sentiment_score 결과 \n{result.get("sentiment_score", "N/A")}\n')
    print(f'language 결과 \n{result.get("language", "N/A")}\n')
    print(f'reply1 결과 \n{result.get("reply1", "N/A")}\n')
    print(f'reply2 결과 \n{result.get("reply2", "N/A")}\n')
except Exception as e:
    print(f"오류 발생: {e}")

translation 결과 
호텔은 깨끗했고 직원들도 매우 친절했습니다. 위치도 편리했으며 주요 관광지와 가까웠어요. 다만, 방이 조금 좁았고 아침 식사 선택지가 제한적이었습니다. 전반적으로 만족스러운 숙박이었지만 개선할 여지가 있습니다.

summary 결과 
깨끗하고 친절한 직원과 편리한 위치가 좋았지만 방이 좁고 아침 식사 선택이 제한된 점이 아쉬웠던 숙박 경험입니다.

sentiment_score 결과 
8

language 결과 
English

reply1 결과 
Thank you very much for sharing your feedback. We truly appreciate your kind words about our cleanliness, friendly staff, and convenient location. We also value your suggestions regarding room size and breakfast options, and we will take them into consideration to improve our services. We hope to have the opportunity to welcome you again and provide an even better experience.

reply2 결과 
소중한 의견을 공유해주셔서 진심으로 감사드립니다. 저희의 청결도, 친절한 직원, 편리한 위치에 대해 좋은 말씀을 해주셔서 매우 감사드립니다. 또한, 객실 크기와 조식 옵션에 관한 소중한 제안들도 감사히 여기며, 더 나은 서비스 제공을 위해 검토하겠습니다. 다시 뵙게 될 기회가 있어 더 나은 경험을 선사할 수 있기를 기대합니다.

